In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory

In [ ]:
MD_PATH = "../data/pdf/test/2106.01345v2(Decision Transformer ReinforcementLearning via Sequence Modeling).md"
API = "groq" # "ollama" or "groq"
# MODEL = "gemma3:270m" # for ollama
MODEL = "llama-3.3-70b-versatile" # for groq

In [ ]:
# 履歴
store = {}
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
# RAG
# 文書読み込み
loader = UnstructuredMarkdownLoader(MD_PATH)
docs = loader.load()
# 文書分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
# ベクトルDB生成
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vectorstore.as_retriever()

In [ ]:
# Create model
if API == "ollama":
    model = ChatOllama(model=MODEL)
elif API == "groq":
    model = ChatGroq(model=MODEL)

In [ ]:
# 履歴を考慮したRAGクエリを生成させるプロンプト
contextualize_q_system_prompt = """
You are given the chat history and the latest user question. Considering these, rephrase the question so that it can be used for searching in a vector store.
If rephrasing is not necessary, use the original question as it is.
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# チャットの履歴を考慮して、ユーザーの質問をより正確な検索クエリに変換
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)

In [ ]:
# 回答生成用のプロンプト（履歴も考慮）
qa_system_prompt = """
You are a professional research assistant in AI.
The following context contains an AI research paper. You should refer to this context when answering the user's questions.

<context>
{context}
</context>
"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# 取得したドキュメントをLLMに渡すためのプロンプトを作成
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)
# 関連ドキュメントを取得し、そのドキュメントと元のユーザーの質問をcreate_stuff_documents_chainに渡して最終的な回答を生成するまでの一連の流れをオーケストレーション
rag_with_history_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
# RunnableWithMessageHistoryでラップして会話形式で実行
conversational_rag_chain = RunnableWithMessageHistory(
    rag_with_history_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
config_rag = {"configurable": {"session_id": "rag_user002"}}

In [ ]:
question = """あなたはプロフェッショナルなAI研究のリサーチアシスタントです。
論文を以下の例のフォーマットでまとめてください。簡条書きかつ、だ・である調で書いてください。
```
### 課題
- AAA
- BBB

### 手法
- AAA
- BBB

### 結果
- AAA
- BBB
```
"""
res = conversational_rag_chain.invoke({"input": question}, config=config_rag)
print(res["answer"])

In [ ]:
question = """手法について以下の例のフォーマットで詳細に解説してください。図があればそれも交えて解説してください。簡条書きかつ、だ・である調で書いてください。
```
### 全体像
提案手法のコンセプトや全体像、全体的なプロセスについて簡潔に解説する。ここで登場する要素技術についてはここでは解説せず、後で詳細に解説する。これらに関する図があればそれも交えて解説する。
- AAA
- BBB

### 要素技術名
提案手法で使用されている要素技術があれば、それにフォーカスして解説する。
- AAA
- BBB

### 要素技術名
提案手法で使用されている要素技術が他にもあれば、それにフォーカスして解説する。
- AAA
- BBB
```

...
"""
res = conversational_rag_chain.invoke({"input": question}, config=config_rag)
print(res["answer"])

In [ ]:
question = """全ての実験結果について以下の例のフォーマットで詳細に解説してください。どのような実験を行ったか、評価指標は何か、どのような結果が得られたかを箇条書きで書いてください。だ・である調で書いてください。
```
### 実験名（例: xxxの性能）
- AAA
- BBB

### 実験名
- AAA
- BBB

...
```
"""
res = conversational_rag_chain.invoke({"input": question}, config=config_rag)
print(res["answer"])

In [ ]:
question = """この研究の課題や展望があれば教えてください。簡条書きかつ、だ・である調で書いてください。
```
### 課題（論文中で述べられている課題があれば述べる）
- AAA
- BBB

### 展望（論文中で述べられている今後の展望があれば述べる）
- AAA
- BBB

### AIの私見（あなたが思う課題や今後の展望があれば述べる）
- AAA
- BBB
```
"""
res = conversational_rag_chain.invoke({"input": question}, config=config_rag)
print(res["answer"])